In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')

import sys
import tempfile
import itertools

import tensorflow as tf

from categorical_embedding_tf_train import loaddata, streaming_mape, input_fn
from categorical_embedding_tf_model import build_estimator

Loading the data:

In [2]:
df_train, df_test = loaddata()

In [3]:
model_dir = "/home/jc/workspace/adidas_take_home/model_dir/"
model_dir = tempfile.mkdtemp() if not model_dir else model_dir

m = build_estimator(model_dir)

INFO:tensorflow:Using config: {'_model_dir': None, '_save_checkpoints_secs': 10, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff428a326d0>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


In [4]:
validation_metrics = {
    "mae":
        tf.contrib.learn.MetricSpec(
            metric_fn=tf.contrib.metrics.streaming_mean_absolute_error,
        ),
    "mape":
        tf.contrib.learn.MetricSpec(
            metric_fn=streaming_mape,
        )
}

validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    input_fn=lambda: input_fn(df_test),
    eval_steps=1,  # Try adding this
    metrics=validation_metrics,
    every_n_steps=10,
    early_stopping_metric="loss",
    early_stopping_metric_minimize=True,
    early_stopping_rounds=10
)

m.fit(
    input_fn=lambda: input_fn(df_train),
    steps=200,
    monitors=[validation_monitor],
)


Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instructions for updating:
Please switch to tf.summary.histogram. Note that tf.summary.histogram uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in.
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instr

DNNLinearCombinedRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x7ff428a32650>, 'linear_optimizer': None, 'embedding_lr_multipliers': None, 'dnn_activation_fn': <function relu at 0x7ff42f6b6230>, 'dnn_optimizer': None, 'joint_linear_weights': False, 'dnn_feature_columns': [_EmbeddingColumn(sparse_id_column=_SparseColumn(column_name='country', is_integerized=False, bucket_size=None, lookup_config=_SparseIdLookupConfig(vocabulary_file=None, keys=('Germany', 'France', 'Austria'), num_oov_buckets=0, vocab_size=3, default_value=-1), combiner='sum', dtype=tf.string), dimension=2, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x7ff452941a10>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, shared_embedding_name=None, shared_vocab_size=None, max_norm=None, trainable=True), _EmbeddingColumn(sparse_id_column=_SparseColumn(column_name='promo1', is_integerized=False, bucket_size=None, lookup_con

After 1800 rounds of training, MAPE reaches 0.288724